In [1]:
"""  copy of ensemble_research for less data to process 
ie. changed elements such as
- tickers
- turbulence_lookback and vol_lookback
- start & end date
- reduced training time ( which has the largest impact on processing time )
- reduced validate and trading time
"""

import os
import pandas as pd
import sys
from pathlib import Path

sys.path.append("..")
from finrl.meta.preprocessor.binancedownloader import BinanceDownloader

In [2]:
tickers = ["SOLUSDT", "XRPUSDT"]

# file to import primary data 
ohlcv_file = "binance_data_raw_example.csv"

# data source path
notebook_path = Path().resolve()   
current_dir = notebook_path.parent
data_path = os.path.join(current_dir, "data")
dt_raw_path = os.path.join(data_path, ohlcv_file)


In [3]:
# retrieve and save the primary ohlcv data

bd = BinanceDownloader()
df_raw = bd.download_multiple(ticker_list=tickers, start_str="1 Jan, 2018")
df_raw.to_csv(dt_raw_path, index=False)

print(f"raw input data saved to...{ohlcv_file}" )

raw input data saved to...binance_data_raw_example.csv


In [4]:
# OR read
df_raw = pd.read_csv(dt_raw_path, parse_dates=["date"])

In [5]:
""" Process data for experiments, and store in data frames (df): 
Exp.1 control-group
Exp.2 voltur-group - 
Exp.3 sen-group - 
"""
from finrl.utils.rolling_windows import get_rolling_windows
from finrl.meta.preprocessor.preprocessors import FeatureEngineer

df_raw = pd.read_csv(dt_raw_path, parse_dates=["date"])

proc_path = os.path.join(data_path, "example_processed_bnc_data.csv")
proc_path_sentiment = os.path.join(data_path, "example_sentiment_bnc_data.csv")
proc_path_voltur = os.path.join(data_path, "example_voltur_bnc_data.csv")


# Exp.1 control-group 
fe = FeatureEngineer(use_technical_indicator=False)
df_processed = fe.preprocess_data(df_raw)
df_processed.to_csv(proc_path, index=False)

# Exp.3 sen-group
fe_sen = FeatureEngineer(use_technical_indicator=False, use_fear_greed=True)
df_sen_processed = fe_sen.preprocess_data(df_raw)
df_sen_processed.to_csv(proc_path_sentiment, index=False)

# Exp.2 voltur-group
from finrl.utils.calculate_turbulence_crypto import add_turbulence_and_volatility
df_voltur_processed = add_turbulence_and_volatility(df_processed, turbulence_lookback=30, vol_lookback=30) 
df_voltur_processed.to_csv(proc_path_voltur, index=False)

print(f"processed and saved input data for Exp.1, Exp.2, and Exp.3" )

/home/derya/finrl_research/notebooks/../finrl/utils/fear_and_greed.py:13: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
/home/derya/finrl_research/notebooks/../finrl/meta/preprocessor/preprocessors.py:176: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["fear_greed"] = df["fear_greed"].fillna(method="ffill")


processed and saved input data for Exp.1, Exp.2, and Exp.3


In [6]:
""" Generate rolling windows """

TRAIN_START_DATE = "2024-05-04"  
TRADE_END_DATE = "2025-05-31"

start_date = pd.Timestamp(TRAIN_START_DATE)
end_date = pd.Timestamp(TRADE_END_DATE)

windows = get_rolling_windows(
    train_months=3,
    val_months=1,
    trade_months=1,
    start_date_str=start_date,
    end_date_str=end_date,
    next_rollings_months= 1,
)

print(f"Generated {len(windows)} rolling windows from {TRAIN_START_DATE} to {TRADE_END_DATE}.")


Created 8 rolling windows.
Generated 8 rolling windows from 2024-05-04 to 2025-05-31.


In [7]:
""" Define Results (daily results) files 
Exp.1 control-group = account_values.csv
Exp.2 voltur-group - sen_account_values.csv (sentiment_account_values)
Exp.3 sen-group - ft_account_values.csv (featured_account_values)
"""
results_path = os.path.join(current_dir, "results")

daily_accounts_path = os.path.join(results_path, "example_account_values.csv")
sen_daily_accounts_path = os.path.join(results_path, "example_sen_account_values.csv")
ft_daily_accounts_path = os.path.join(results_path, "example_ft_account_values.csv")



In [8]:
"""
"""
from finrl.config import model_configs

PPO_model_kwargs = model_configs["PPO"]
A2C_model_kwargs = model_configs["A2C"]
DDPG_model_kwargs = model_configs["DDPG"]



In [9]:
from finrl.utils.compute_sharpe_metrics import compute_sharpe_metrics
from stable_baselines3.common.vec_env import DummyVecEnv
from finrl.meta.preprocessor.preprocessors import data_split
from finrl.meta.env_crypto_trading.env_cryptotrading import CryptoTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
import numpy as np

experiments = [
   {"df": df_processed, "result_fl": daily_accounts_path, "featured":False, "sentiment":False},
   {"df": df_voltur_processed, "result_fl": ft_daily_accounts_path, "featured":True, "sentiment":False},
   {"df": df_sen_processed, "result_fl": sen_daily_accounts_path, "featured":False, "sentiment":True}
]

# for each experiment
for j, exp in enumerate(experiments):

    # reset experiement's results data frame
    account_values_df =  pd.DataFrame()
    
  #  exp = experiments[j]
    input_data = exp["df"]
    results_fl = exp["result_fl"] 
    is_featured = exp["featured"]
    is_sentiment =  exp["sentiment"]

    # for each window 1 to 22
    for i, (train_start, train_end, val_start, val_end, trade_start, trade_end) in enumerate(windows):
        
        # split data for training, validation and testing (trade)
        train_data = data_split(input_data, train_start, train_end)
        val_data = data_split(input_data, val_start, val_end)
        trade_data = data_split(input_data, trade_start, trade_end)

        window_name = f"window_{i+1}"
        print(f" Rolling {window_name}: {train_start.date()} to {trade_end.date()}")
        
        # Train all 3 models
        env_train = DummyVecEnv([lambda: CryptoTradingEnv(train_data, featured=is_featured, sentiment=is_sentiment)])
        agent = DRLAgent(env=env_train)
    
        models = {
            "ppo": agent.train_PPO(total_timesteps=len(train_data)*30, model_kwargs=PPO_model_kwargs),
            "a2c": agent.train_A2C(total_timesteps=len(train_data)*30, model_kwargs=A2C_model_kwargs),
            "ddpg": agent.train_DDPG(total_timesteps=int(len(train_data)*30*0.5), model_kwargs=DDPG_model_kwargs)
        }
    
        #----------------------------------------------------------------------------------------------------------------------
        # Validation : Select best model based on sharp produced during validation
        # Note: daily trading results, ie dates, account value, daily return etc are not recorded during validation
        # only sharp is calculated to select the best model
        #----------------------------------------------------------------------------------------------------------------------
        
        best_model = None
        best_sharpe = -np.inf
        val_sharpes = []
        
        for name, model in models.items():
            
            env_val = DummyVecEnv([lambda: CryptoTradingEnv(val_data, featured=is_featured, sentiment=is_sentiment)])
            sharpe_metrics = DRLAgent.DRL_evaluation(model=model, environment=env_val)
            sharpe = sharpe_metrics["sharpe"]
            val_sharpes.append({"name": name, "sharpe": sharpe})
    
            if not np.isnan(sharpe) and sharpe > best_sharpe:
                best_model = model
                best_sharpe = sharpe    
    
        #----------------------------------------------------------------------------------------------------------------------
        # Trade using the best model, record results  
        # basic data (for control) recorded daily includes:   "date",  "account_value", "daily_return", "daily_volatility" 
        #----------------------------------------------------------------------------------------------------------------------
               
        if best_model is not None:
            env_trade = DummyVecEnv([lambda: CryptoTradingEnv(trade_data, featured=is_featured, sentiment=is_sentiment)])

            startdt = pd.to_datetime(trade_start)
            env_trade.envs[0].trading_mode = True
            df_result = DRLAgent.DRL_prediction(model=best_model, environment=env_trade, start_date=startdt)

            df_result["window"] = i + 1

             # concat for reporting
            if "account_values_df" not in locals():
                account_values_df = df_result.copy()
            else:
                account_values_df = pd.concat([account_values_df, df_result], ignore_index=True)
          
        else:
            print(f"❌ No valid model selected in window {i+1} — skipping trade step.")

            #  --- Trade End ---
    
        print(f"✅ Window {i+1} Summary:")
        print(f"  🔹 Train Dates : {train_start.date()} → {train_end.date()}")
        print(f"  🔹 Trade Dates : {trade_start.date()} → {trade_end.date()}")
        print(f"  🔹 Best Model  : {best_model.__class__.__name__ if best_model else 'None'}")
        print(f"  🔹 Sharpe      : {best_sharpe:.2f}")


    
    # get daily performence file name for the experiment j and save to dedicated location
     
    account_values_df.to_csv(results_fl, index=False)
    print(f"Daily account values saved to", {results_fl})


for exp 0 is_featured:False
for exp 0 is_sentiment:False
 Rolling window_1: 2024-05-04 to 2024-10-03
[EVAL] Sharpe Debug: No volatility or insufficient data.
✅ Window 1 Summary:
  🔹 Train Dates : 2024-05-04 → 2024-08-03
  🔹 Trade Dates : 2024-09-04 → 2024-10-03
  🔹 Best Model  : PPO
  🔹 Sharpe      : 0.45
 Rolling window_2: 2024-06-04 to 2024-11-03
[EVAL] Sharpe Debug: No volatility or insufficient data.
✅ Window 2 Summary:
  🔹 Train Dates : 2024-06-04 → 2024-09-03
  🔹 Trade Dates : 2024-10-04 → 2024-11-03
  🔹 Best Model  : A2C
  🔹 Sharpe      : 0.29
 Rolling window_3: 2024-07-04 to 2024-12-03
[EVAL] Sharpe Debug: No volatility or insufficient data.
✅ Window 3 Summary:
  🔹 Train Dates : 2024-07-04 → 2024-10-03
  🔹 Trade Dates : 2024-11-04 → 2024-12-03
  🔹 Best Model  : PPO
  🔹 Sharpe      : 0.48
 Rolling window_4: 2024-08-04 to 2025-01-03
[EVAL] Sharpe Debug: No volatility or insufficient data.
✅ Window 4 Summary:
  🔹 Train Dates : 2024-08-04 → 2024-11-03
  🔹 Trade Dates : 2024-12-04 →